# Kaggle Categorical Feature Competition <br>
The competition is setup to practice skills working with binary, ordinal, and nominal data.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [25]:
import pandas as pd
sample_submission = pd.read_csv("cat-in-the-dat-ii/sample_submission.csv")
test = pd.read_csv("cat-in-the-dat-ii/test.csv",index_col = 'id')
train = pd.read_csv("cat-in-the-dat-ii/train.csv",index_col = 'id')

In [26]:
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,...,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,...,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,...,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,...,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,...,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [27]:
import pandas_profiling
pandas_profiling.ProfileReport(train)

In [3]:
#define the data sets
#remove rows with missing target
train.dropna(axis=0, subset=['target'], inplace=True)
train_y=train.target
train.drop(['target'],axis=1, inplace=True)

In [4]:
#comparing the unique results in the train and test dataset.  Replacing values that do not 
#appear in the other set with value, 'xor'
for col in train.columns.difference(["id"]):
    train_vals = set(train[col].dropna().unique())
    test_vals = set(test[col].dropna().unique())

    xor_cat_vals = train_vals ^ test_vals
    if xor_cat_vals:
        print(f"Replacing {len(xor_cat_vals)} values in {col}, {xor_cat_vals}")
        train.loc[train[col].isin(xor_cat_vals), col] = "xor"
        test.loc[test[col].isin(xor_cat_vals), col] = "xor"

Replacing 1 values in nom_5, {'b3ad70fcb'}
Replacing 4 values in nom_6, {'ee6983c6d', 'a885aacec', '3a121fefb', 'f0732a795'}
Replacing 2 values in nom_9, {'1065f10dd', '3d19cd31d'}


In [125]:
'''Variable Description'''
def description(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values
    summary['PercMissing'] = df.isnull().sum().values / df.isnull().count().values
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary
print('**Variable Description of  train Data:**')
description(train)

**Variable Description of  train Data:**
Dataset Shape: (600000, 23)


,Name,dtypes,Missing,PercMissing,Uniques,First Value,Second Value,Third Value
0,bin_0,float64,17894,0.029823,2,0,1,0
1,bin_1,float64,18003,0.030005,2,0,1,1
2,bin_2,float64,17930,0.029883,2,0,0,0
3,bin_3,object,18014,0.030023,2,F,F,F
4,bin_4,object,18047,0.030078,2,N,Y,N
5,nom_0,object,18252,0.030420,3,Red,Red,Red
6,nom_1,object,18156,0.030260,6,Trapezoid,Star,NaN
7,nom_2,object,18035,0.030058,6,Hamster,Axolotl,Hamster
8,nom_3,object,18121,0.030202,6,Russia,NaN,Canada
9,nom_4,object,18035,0.030058,4,Bassoon,Theremin,Bassoon


In [5]:
#Fill in the missing values with mode
columns = train.columns
for c in columns:
    train[c].fillna(train[c].mode()[0],inplace=True)
    test[c].fillna(test[c].mode()[0],inplace=True)


In [6]:
#list the columns to be used for label encoding
columns_to_ord = ['bin_3','bin_4','ord_1', 'ord_2']
for col in columns_to_ord:
    print('{}'.format(col), train[col].unique(),'\n' )


bin_3 ['F' 'T'] 

bin_4 ['N' 'Y'] 

ord_1 ['Contributor' 'Grandmaster' 'Novice' 'Expert' 'Master'] 

ord_2 ['Hot' 'Warm' 'Freezing' 'Lava Hot' 'Cold' 'Boiling Hot'] 



In [7]:
#Create the dictionaries used to replace the string value with an integer value
dict_bin_3 = {'T':1, 'F':0}
dict_bin_4 = {'Y': 1, 'N':0}
dict_ord_1 = {'Novice':1,'Contributor':2,'Expert':3,'Master':4,'Grandmaster':5, }
dict_ord_2 = {'Freezing':1,'Cold':2,'Warm':3,'Hot':4,'Boiling Hot':5,'Lava Hot':6}

In [8]:
#replace both the train and test data
dict_list=[dict_bin_3, dict_bin_4,dict_ord_1,dict_ord_2]
cols = ['bin_3','bin_4','ord_1', 'ord_2']
for i in range(0,len(cols)):
    col = cols[i]
    d = dict_list[i]
    train[col]=train[col].replace(d)
    test[col]=test[col].replace(d)

In [9]:
#Change month and day to string and category encode below
train.month=train.month.apply(lambda x: str(x))
train.day=train.day.apply(lambda x: str(x))
test.month=test.month.apply(lambda x: str(x))
test.day=test.day.apply(lambda x: str(x))

In [10]:
cat_feat_to_encode=list(train.select_dtypes(include='object').columns)

In [12]:
import category_encoders as ce
smoothing = 0.2
oof = pd.DataFrame([])
from sklearn.model_selection import StratifiedKFold
for tr_idx, oof_idx in StratifiedKFold(n_splits=5, random_state=2020, shuffle=True).split(train, train_y):
    ce_target_encoder = ce.TargetEncoder(cols = cat_feat_to_encode, smoothing=smoothing)
    ce_target_encoder.fit(train.iloc[tr_idx, :], train_y.iloc[tr_idx])
    oof = oof.append(ce_target_encoder.transform(train.iloc[oof_idx, :]), ignore_index=False)
ce_target_encoder = ce.TargetEncoder(cols = cat_feat_to_encode, smoothing=smoothing)
ce_target_encoder.fit(train, train_y);  train = oof.sort_index(); test = ce_target_encoder.transform(test)

In [13]:
#Create the data set to train and test the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, train_y, test_size=0.3, random_state=42)

Testing Random Forest Classifier and performing parameter selection.

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [16]:
rf6 = RandomForestClassifier()
params = {
    'max_features': [0.4, 5, 10],
    'n_estimators': [100, 200],
    'min_samples_leaf': [1,2],
    'random_state':[42],
}
cv = GridSearchCV(
rf6, params, n_jobs=-1).fit(X_train, y_train)
print(cv.best_params_)

{'max_features': 0.4, 'min_samples_leaf': 2, 'n_estimators': 200, 'random_state': 42}


Copy the original cleaned data set and process using resample.  The target column is weighted on the target = 0 side.  Resampling will create an even distributed of 1's and 0's.

In [17]:
dffull = pd.concat([train, train_y])

In [24]:
train_y['target']=train_y

In [13]:
data=train.copy()
data['target']=train_y
data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0,0,0.183161,0.224364,0.169130,0.219119,0.208137,...,0.158824,3.0,2,4,0.114332,0.223733,0.145339,0.200839,0.146486,0
1,1.0,1.0,0.0,0,1,0.183161,0.130122,0.201132,0.165718,0.179435,...,0.143204,3.0,5,3,0.134617,0.234159,0.237785,0.212760,0.208616,0
2,0.0,1.0,0.0,0,0,0.183161,0.159930,0.169130,0.158044,0.208137,...,0.190030,3.0,1,1,0.269553,0.200635,0.214833,0.163350,0.211561,0
3,0.0,0.0,0.0,0,0,0.183307,0.179869,0.170040,0.179773,0.179714,...,0.262295,1.0,1,6,0.092533,0.138102,0.165279,0.164827,0.146925,0
4,0.0,0.0,0.0,1,0,0.183272,0.160088,0.169792,0.202740,0.179884,...,0.251641,3.0,5,2,0.181226,0.140080,0.165629,0.162757,0.224777,0


In [14]:
from sklearn.utils import resample
mask = data.target == 1
yes_df = data[mask]
no_df = data[~mask]
df_upsample = resample(
yes_df, 
replace=True,
n_samples=len(no_df),
random_state = 42,
)
df2 = pd.concat([no_df, df_upsample])
df2.target.value_counts()

1    487677
0    487677
Name: target, dtype: int64

# Comparing the original dataset with the resampled data set.

In [15]:
df2_y=df2.target
df2.drop(['target'],axis=1, inplace=True)
df2_y=df2_y.astype('int64')

In [19]:
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(df2, df2_y, test_size=0.3, random_state=42)

The original cleaned data is X_train, X_test, y_train, y_test<br>
The resampled is X_train2, X_test2, y_train2, y_test2.<br>
Will run a simple Random Forest Regressor and compare results.

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf_1=RandomForestClassifier(n_estimators = 100, random_state=42)
rf_1.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [37]:
rf_1.score(X_test,y_test)


0.8206166666666667

In [38]:
from sklearn import metrics
metrics.precision_score(y_test, rf_1.predict(X_test))

0.6371481405383193

In [64]:
rf_2=RandomForestClassifier(n_estimators = 100, random_state=42)
rf_2.fit(X_train2, y_train2)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [65]:
rf_2.score(X_test2,y_test2)

0.9491844009200053

In [66]:
metrics.precision_score(y_test2, rf_2.predict(X_test2))

0.9340218736147041

The resampled data performed better.  Run the Random Forest Classifier model to search for best params.<br>  Then use these to make a submission.  

In [68]:
rf7 = RandomForestClassifier()
params = {
    'max_features': [0.4,1,'auto'],
    'n_estimators': [100, 200],
    'min_samples_leaf': [1,2],
    'random_state':[42],
}
cv = GridSearchCV(
rf7, params, n_jobs=-1).fit(X_train2, y_train2)
print(cv.best_params_)

{'max_features': 1, 'min_samples_leaf': 1, 'n_estimators': 200, 'random_state': 42}


In [70]:
model_rf_resx = RandomForestClassifier(max_features = 1, min_samples_leaf = 1, 
                                      n_estimators = 200, random_state=42)
model_rf_resx.fit(df2, df2_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=1,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [71]:
pred_rf_resx=model_rf_resx.predict(test)

In [72]:
output=pd.DataFrame({'Id':test.index, 'Target':pred_rf_resx})
output.to_csv('submission8.csv', index=False)

In [73]:
from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import roc_auc_score

model_rf_r = RandomForestRegressor(n_estimators = 100, random_state=0)
model_rf_r.fit(df2, df2_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [74]:
pred_rf_r=model_rf_r.predict(test)

In [75]:
output=pd.DataFrame({'Id':test.index, 'Target':pred_rf_r})
output.to_csv('submission9.csv', index=False)

Random Forest Regressor with the resampled data gave better results on the Kaggle leaderboard than the Random Forest Classifier.  

Rerunning Random Forest Regressor with n_estimators = 200 to further improve score.

In [22]:
from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import roc_auc_score

model_rf_r2 = RandomForestRegressor(n_estimators = 200, random_state=0)
model_rf_r2.fit(df2, df2_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [23]:
pred_rf_r2=model_rf_r2.predict(test)

In [24]:
output=pd.DataFrame({'Id':test.index, 'Target':pred_rf_r2})
output.to_csv('submission10.csv', index=False)